In [1]:
import sys

sys.path.append("/Users/ohadr/chains")
from src.chain import MessageChain

/Users/ohadr/.pyenv/versions/3.10.10/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [7]:
ref_goap = open("/Users/ohadr/egbench/goap/code/ref_goap.py", "r").read()
sensor_goap = open("/Users/ohadr/egbench/goap/code/sensor_goap.py", "r").read()

In [ ]:
content = f"""

You will be given two files, each implementing a different version of GOAP.
The first file is an implementation of GOAP that allows referencing the effects of actions,
and the second file is a more complex implementation of GOAP.

The first file is a simple implementation of GOAP, and the second file is a more complex implementation of GOAP.



{ref_goap}"""

with open("prompt.txt", "w") as f:
    f.write(content)

In [6]:
content = f"""You will be a file that implements a GOAP system.
It is a specialized version of GOAP that allows referencing the effects of actions,
Your goal is to convert the code into a specification that would prompt someone to implement the *exact* system.
It is part of a homework assignment for students in a game programming course.

The quality of your specification will be evaluated by how well it can be used to implement the *exact* system, meaning that if you omit any details, the implementation will not be able to replicate the original system.



```
{ref_goap}
```

Reminder:
Your goal is to convert the code into a specification that would prompt someone to implement the *exact* system.
It is part of a homework assignment for students in a game programming course.

The quality of your specification will be evaluated by how well it can be used to implement the *exact* system, meaning that if you omit any details, the implementation will not be able to replicate the original system.
"""

with open("prompt_for_spec_ref.txt", "w") as f:
    f.write(content)

In [8]:
content = f"""You will be a file that implements a GOAP system.
It is a specialized version of GOAP that allows sensors and working memory to be used by the planner.
Your goal is to convert the code into a specification that would prompt someone to implement the *exact* system.
It is part of a homework assignment for students in a game programming course.

The quality of your specification will be evaluated by how well it can be used to implement the *exact* system, meaning that if you omit any details, the implementation will not be able to replicate the original system.



```
{sensor_goap}
```

Reminder:
Your goal is to convert the code into a specification that would prompt someone to implement the *exact* system.
It is part of a homework assignment for students in a game programming course.

The quality of your specification will be evaluated by how well it can be used to implement the *exact* system, meaning that if you omit any details, the implementation will not be able to replicate the original system.
"""

with open("prompt_for_sensor_ref.txt", "w") as f:
    f.write(content)

In [80]:
both_spec = open("/Users/ohadr/egbench/goap/code/both_spec.md", "r").read()

import numpy as np


# Shuffle the list
import re
def remove_enum(text):
    # Match number + dot or parenthesis at start of string only
    pattern = r'^\d+[.)]'
    return re.sub(pattern, '', text)


def conv_into_sections(text):
    text = text.split("\n\n\n")
    text = [remove_enum(x.strip()).strip() for x in text if x.strip()]
    text = [x for x in text if x.strip("”").strip()]
    return text


def shuffle_write(sections):
    shuffled_spec = sections.copy()
    np.random.seed(42)
    np.random.shuffle(shuffled_spec)
    # Add numbers and store in new list

    import tempfile
    with tempfile.NamedTemporaryFile(mode='w', delete=False) as f:
        f.write("\n".join([f"Section {i+1}\n{item}\n" for i, item in enumerate(shuffled_spec)]))
        print(f"Wrote shuffled sections to temporary file: {f.name}")
    return shuffled_spec
both_spec = conv_into_sections(both_spec)
shuffled_spec = shuffle_write(both_spec)

Wrote shuffled sections to temporary file: /var/folders/7k/w0ptgvjx7m1gvprjz4wdtnm00000gn/T/tmph6nr8pyc


In [31]:
ordering = [1, 3, 2, 16, 4, 5, 14, 6, 7, 9, 10, 11, 12, 15, 13, 17, 18, 8]
ordered_spec = [shuffled_spec[i - 1] for i in ordering]

with open("both_spec_ord.md", "w") as f:
    f.write("\n\n".join(ordered_spec))

In [36]:
both_spec1 = """

INTRODUCTION  
“Your task is to implement a regressive Goal-Oriented Action Planning (GOAP) system in Python that exactly matches the features and behaviors described below. The system must support references—i.e., dynamic data passed from one action’s effect into the preconditions of subsequent actions—even if the actual value is only determined at runtime. By defining simple, isolated actions—and letting the system figure out the chain of dependencies—you can rapidly prototype sophisticated behaviors that respond to emergent gameplay changes.”
“This specification describes a specialized Goal-Oriented Action Planning (GOAP) system that incorporates sensors and working memory, and a state machine to distribute computation and drive AI behavior in real time. Implement the entire code architecture described here to replicate the system exactly, meaning that if you omit any details, the implementation will not be able to replicate the original system.”


1. TERMINOLOGY AND HIGH-LEVEL STRUCTURE  
“An agent develops a plan in real time by supplying a goal to satisfy a planner. The GOAP planner looks at an action’s preconditions and effects in order to determine a queue of actions to satisfy the goal. For a full deep dive into the code itself, please refer to the listing provided in this post’s code blocks.”
“This system is a flexible AI planning system that allows agents to determine their own sequence of actions. The system must support referencing the effects of actions, meaning that an action’s preconditions can use dynamic data produced by another action’s effect.”

2. REQUIRED DATA STRUCTURES AND TYPES  
“A set of possible goals (conditions to fulfill), a set of actions (each with preconditions and effects), a world state (the environment and the agent’s own data), sensors that detect changes in the world, and a working memory storing perceptions in a consistent format. Facts could be hashed into bins based on the type of knowledge, or sorted in some manner.”
“The system must define a dictionary representing the current world state, a collection of Action objects, each with symbolic preconditions/effects, references to pass data, and classes such as WorkingMemoryFact for storing knowledge from sensors.”

3. HOW REFERENCES WORK (EffectReference, reference(..))  
“References allow you to ‘pass forward’ an effect from one action into the preconditions of subsequent actions, even if the actual value is only determined at runtime. This is done via class EffectReference: A small class that annotates a requested effect name—and def reference(name: str): A convenience function that returns an EffectReference referencing that name.”
“For instance, if Action A has effects = { "tool_id": ... }, it means ‘I will provide a tool_id (but we’ll only know the real value at runtime).’ Another action B can specify preconditions = { "required_tool_id": reference("tool_id") }, meaning ‘I depend on the tool_id that was created or identified by the earlier action.’”

4. SENSORS AND WORKING MEMORY (WorkingMemoryFact)  
“All knowledge generated by sensors is stored in working memory in a common format. Sensors deposit perceptions in working memory and the planner uses these perceptions to guide its decision-making. A WorkingMemoryFact is a record containing a set of associated attributes. Different subsets of attributes are assigned depending on the type of knowledge the fact represents.”

“Sensors perceive external visible and audible stimuli, as well as internal stimuli such as pain and desires. Some sensors are event-driven while others poll. Event-driven sensors are useful for recognizing instantaneous events like sounds and damage. Polling works better for sensors that need to extract information from the world.”

5. FIVE DISTINCT SCENARIOS THAT USE REFERENCES (OPTIONAL)  
“Below are five hypothetical or illustrative usage scenarios that demonstrate how you can harness references in actions. These scenarios do not include the chanting example from the code snippet—thus giving fresh, concrete insight into references. For instance, passing a tool ID, flipping a table for cover, picking up a key to unlock a door, linking a generated power-on state, or using a repaired bridge to cross a river.”


"""
conv_into_sections(both_spec1)

['The system must support references—i.e., dynamic data passed from one action’s effect into the preconditions of subsequent actions—even if the actual value is only determined at runtime. By defining simple, isolated actions—and letting the system figure out the chain of dependencies—you can rapidly prototype sophisticated behaviors that respond to emergent gameplay changes.”\n“This specification describes a specialized Goal-Oriented Action Planning (GOAP) system that incorporates sensors and working memory, and a state machine to distribute computation and drive AI behavior in real time. Implement the entire code architecture described here to replicate the system exactly, meaning that if you omit any details, the implementation will not be able to replicate the original system.”',
 'TERMINOLOGY AND HIGH-LEVEL STRUCTURE  \n“An agent develops a plan in real time by supplying a goal to satisfy a planner. The GOAP planner looks at an action’s preconditions and effects in order to determ

In [39]:
please suggest a coherent ordering of the following (just output numbers). think first. you should give a permutation of the numbers 1-25.



SyntaxError: invalid syntax (119269158.py, line 1)

In [48]:
# len()
shuffled_spec2 = shuffle_write(list(set(ordered_spec + conv_into_sections(both_spec1))))
print(len(shuffled_spec2))
shuffled_spec2

Wrote shuffled sections to temporary file: /var/folders/7k/w0ptgvjx7m1gvprjz4wdtnm00000gn/T/tmp7ptwtd46
20


['REQUIRED DATA STRUCTURES AND TYPES  \n“A set of possible goals (conditions to fulfill), a set of actions (each with preconditions and effects), a world state (the environment and the agent’s own data), sensors that detect changes in the world, and a working memory storing perceptions in a consistent format. Facts could be hashed into bins based on the type of knowledge, or sorted in some manner.”',
 'SENSORS, FACTS, AND WORKING MEMORY  \na) Sensor Interface  \n• Each sensor has an update method: sense(memory, worldState).  \n• On invocation, the sensor inspects the world (or worldState) and generates facts.  \nb) Fact Objects  \n• A fact contains, at minimum:  \n  – factType (e.g., “EnemyLocation” or “DoorState”).  \n  – data (could be a dictionary for more attributes).  \n  – confidence (a float from 0 to 1).  \n• The memory class stores and retrieves facts, possibly removing old ones as needed.  \nc) Planner Integration  \n• Before or during planning, the AI consults working memory

In [42]:
ordering2 = [24, 8, 3, 14, 11, 1, 9, 10, 4, 22, 6, 2, 16, 23, 15, 12, 13, 17, 5, 7, 18, 19, 20, 21, 25]

In [46]:
ordered_spec = [shuffled_spec2[i - 1] for i in ordering2]

with open("both_spec_ord2.md", "w") as f:
    f.write("\n\n".join(ordered_spec))

In [45]:
len(ordering)

18

In [73]:
len("────────────────────────────────────────────────────────────────────")

68

In [81]:
both_critique_spec = open(
    "/Users/ohadr/egbench/goap/code/both_critique_spec.txt", "r"
).read()
sensor_spec = open("/Users/ohadr/egbench/goap/code/sensor_spec.txt", "r").read()
ref_spec = open("/Users/ohadr/egbench/goap/code/ref_spec.txt", "r").read()
specs = conv_into_sections(both_critique_spec)+ conv_into_sections(ref_spec) + conv_into_sections(
    sensor_spec.replace(
        "────────────────────────────────────────────────────────────────────", "\n\n\n"
    )
)

In [87]:
# len()
# please suggest a coherent ordering of the following (just output numbers). think first. you should give a permutation of the numbers 1-58.


shuffled_spec3 = shuffle_write(specs)
print(len(shuffled_spec3))
# shuffled_spec3

Wrote shuffled sections to temporary file: /var/folders/7k/w0ptgvjx7m1gvprjz4wdtnm00000gn/T/tmphc9n1git
58


In [85]:
ordering3 = [1, 8, 40, 26, 20, 28, 53, 12, 27, 42, 50, 51, 13, 14, 21, 31, 35, 38, 56, 2, 17, 25, 29, 37, 46, 54, 11, 18, 22, 23, 55, 57, 3, 24, 32, 39, 45, 48, 49, 9, 15, 41, 44, 47, 52, 4, 5, 6, 7, 10, 16, 19, 30, 33, 34, 36, 43, 58]

In [86]:
ordered_spec3 = [shuffled_spec3[i - 1] for i in ordering3]

with open("both_spec_ord3.md", "w") as f:
    f.write("\n\n".join(ordered_spec3))